In [3]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING = dataiku.Dataset("MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING")
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING_df = MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING.get_dataframe()

MATCHES_1_TO_1_STAGING = dataiku.Dataset("MATCHES_1_TO_1_STAGING")
MATCHES_1_TO_1_STAGING_df = MATCHES_1_TO_1_STAGING.get_dataframe()

In [5]:
df_1_N = MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING_df
print(len(df_1_N))
df_1_N.head()

106


,CUSTOMER,MATCH_CUSTOMER,DRAW_UP_DATE
0,130199-BROWN & CALDWELL,"['BROWN AND CALDWELL', 'CALDWELL ELECTRICAL CO...","[Timestamp('2021-01-01 00:00:00+0000', tz='UTC..."
1,ZORN COMPRESSOR & EQUIPMENT IN,"['METROPOLITAN AIR COMPRESSOR CO INC', 'NATION...","[Timestamp('2019-02-01 00:00:00+0000', tz='UTC..."
2,WORLDWIDE WINE & SPIRITS INC,"['FISHER59 OKLAHOMA WINE SPIRITS LLP', 'WORLD...","[Timestamp('2019-11-01 00:00:00+0000', tz='UTC..."
3,WEST YAVAPAI GUIDANCE CLINI (2,['DELAWARE GUIDANCE SERVICES FOR CHILDREN AND ...,"[Timestamp('2019-04-01 00:00:00+0000', tz='UTC..."
4,WAVE FORM SYSTEMS INC,"['CONCRETE FORM ERECTORS INC', 'FORM WAVE SYST...","[Timestamp('2019-02-01 00:00:00+0000', tz='UTC..."


In [6]:
df_1_1 = MATCHES_1_TO_1_STAGING_df
print(len(df_1_1))
df_1_1.head()

1154


,CUSTOMER,MATCH_CUSTOMER,DRAW_UP_DATE
0,0972 HEINEKEN USA INC (PHH35),0972 HEINEKEN USA INC (PHH47),2019-06-01 00:00:00+00:00
1,ZOOM SEWER & DRAIN CLEANING SE,ZOOM SEWER AND DRAIN CLEANING SERVICE INC,2019-05-01 00:00:00+00:00
2,ZERNCO INC,ZERNCO,2020-07-01 00:00:00+00:00
3,ZASTROW THE BEER MAN,ZASTROW THE BEER MAN INC,2021-10-01 00:00:00+00:00
4,YOUR WAY FUMIGATION INC,FUMIGATION SERVICE SUPPLY INC,2022-02-01 00:00:00+00:00


In [19]:
from difflib import SequenceMatcher
import Levenshtein

idx = 0
exception_count = 0

_verified = []
_verified_match = []

_exception = []
_exception_match = []

for index, row in df_1_1.iterrows():

    idx+=1
    
    customer = row['CUSTOMER']
    match_customer = row['MATCH_CUSTOMER']
    
    r = Levenshtein.ratio(customer, match_customer)
    if r<0.8:
        _exception.append(customer)
        _exception_match.append(match_customer)
    else:
        _verified.append(customer)
        _verified_match.append(match_customer)

In [20]:
for i in range(0, len(_verified)):
    
    print(_verified[i], _verified_match[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

#MATCHES_VERIFIED_df = ... # Compute a Pandas dataframe to write into MATCHES_VERIFIED


# Write recipe outputs
#MATCHES_VERIFIED = dataiku.Dataset("MATCHES_VERIFIED")
#MATCHES_VERIFIED.write_with_schema(MATCHES_VERIFIED_df)